# 1、关于对话模型中消息（message）的使用

标准的对话模型的调用过程

invoke()的输入可以是多种类型，典型的类型有：1.字符串类型 2.消息列表

invoke()的输出类型：BaseMessage的子类：AIMessage

In [1]:
from langchain_openai import ChatOpenAI
import os
import dotenv

# 前提，加载配置文件
dotenv.load_dotenv()

# 获取对话模型
chat_model = ChatOpenAI(
    model="qwen3-max",
    temperature=0,
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
    api_key=os.getenv("DASHSCOPE_API_KEY"),
)

# 调用对话模型
response = chat_model.invoke("帮我解释一下什么是langchain？")

# 处理响应数据
print(response)



content='LangChain 是一个开源框架，旨在简化使用大型语言模型（LLMs，如 GPT、Llama、Claude 等）构建应用程序的过程。它的核心目标是让开发者能够轻松地将语言模型与外部数据源、工具和逻辑结合起来，从而创建更强大、更智能的应用程序，而不仅仅是依赖模型自身的“静态”知识。\n\n### LangChain 的主要特点包括：\n\n1. **模块化设计**  \n   LangChain 提供了一系列可组合的组件（如提示模板、记忆机制、链式调用、代理等），开发者可以像搭积木一样将它们组合起来，快速构建复杂应用。\n\n2. **支持多种语言模型**  \n   它兼容主流的 LLM 提供商（如 OpenAI、Anthropic、Hugging Face、Google、Meta 等），让你可以轻松切换或集成多个模型。\n\n3. **连接外部数据（RAG）**  \n   通过“检索增强生成”（Retrieval-Augmented Generation, RAG）技术，LangChain 可以让模型访问你自己的私有数据（如文档、数据库、API 等），从而回答基于最新或特定领域信息的问题。\n\n4. **链（Chains）与代理（Agents）**  \n   - **Chains**：将多个步骤串联起来（例如：先获取用户输入 → 查询数据库 → 生成回答）。\n   - **Agents**：让 LLM 能够自主决定调用哪些工具（如搜索、计算器、代码执行等）来完成任务，实现更灵活的推理。\n\n5. **记忆（Memory）机制**  \n   支持在对话中保留上下文（比如聊天历史），使交互更连贯自然。\n\n6. **生态系统丰富**  \n   LangChain 拥有活跃的社区和大量示例，涵盖问答系统、聊天机器人、自动化助手、数据分析等多种应用场景。\n\n---\n\n### 举个简单例子：\n你想做一个能回答公司内部文档问题的聊天机器人。使用 LangChain，你可以：\n- 将公司文档加载并切分成片段；\n- 用向量数据库（如 FAISS 或 Pinecone）存储这些片段；\n- 当用户提问时，系统先从数据库中检索相关文档；\n- 再把问题和检索到的内容一起交给 LLM 生成答案。\n\n整个流程可以用几行 LangCha

关于消息：

有典型的三类消息：SystemMessage、HumanMessage、AIMessage

In [2]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

system_message = SystemMessage(content="你是一个有帮助的助手。")
human_message = HumanMessage(content="请告诉我今天的天气如何？")

messages = [system_message, human_message]

print(messages)

[SystemMessage(content='你是一个有帮助的助手。', additional_kwargs={}, response_metadata={}), HumanMessage(content='请告诉我今天的天气如何？', additional_kwargs={}, response_metadata={})]


使用大模型，调用消息列表

In [3]:
response = chat_model.invoke(messages)
print(type(response))
print(response.content)

<class 'langchain_core.messages.ai.AIMessage'>
我无法获取实时天气信息。建议您查看当地的天气预报网站、使用天气应用程序（如“天气通”、“墨迹天气”或手机自带的天气功能），或在搜索引擎中输入“您所在城市 + 今天天气”来获取最新天气情况。如果您告诉我您所在的城市，我可以指导您如何查找相关信息。


In [4]:
from langchain_core.messages import SystemMessage, HumanMessage

system_message = SystemMessage(
    content="你是一个有帮助的助手。",
    additional_kwargs={"tool": "invoke_func1"},
)
human_message = HumanMessage(content="请告诉我今天的天气如何？")

messages = [system_message, human_message]

print(messages)

[SystemMessage(content='你是一个有帮助的助手。', additional_kwargs={'tool': 'invoke_func1'}, response_metadata={}), HumanMessage(content='请告诉我今天的天气如何？', additional_kwargs={}, response_metadata={})]


# 2、关于多轮对话与上下文记忆

In [5]:
from langchain_openai import ChatOpenAI
import os
import dotenv

# 前提，加载配置文件
dotenv.load_dotenv()

# 获取对话模型
chat_model = ChatOpenAI(
    model="qwen3-max",
    temperature=0,
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
    api_key=os.getenv("DASHSCOPE_API_KEY"),
)

测试1：

In [6]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

system_message = SystemMessage(content="我是一个人工智能助手，我的名字叫小智。")
human_message = HumanMessage(content="猫王是一只猫吗？")

messages = [system_message, human_message]

response = chat_model.invoke(messages)
print(response.content)

response1 = chat_model.invoke("你叫什么名字？")
print(response1.content)

哈哈，这是个很有趣的问题！

“猫王”通常并不是指一只真正的猫，而是对美国著名摇滚歌手**埃尔维斯·普雷斯利**（Elvis Presley）的昵称。他因为开创性的音乐风格、独特的舞台魅力和标志性的发型与舞步，被誉为“摇滚乐之王”，中文里常亲切地称他为“猫王”。

不过，“猫王”这个词从字面上看确实像是“猫中的王者”，所以也经常被用来给宠物猫起名字，或者出现在一些以猫为主角的搞笑、可爱语境中。比如有人会说：“我家主子是猫王！”——这时候就真的在说一只猫啦！

所以总结一下：

- **通常意义上**：“猫王” = 埃尔维斯·普雷斯利（人，不是猫）。
- **字面或趣味用法**：也可能真的指一只特别霸气或可爱的猫！

你是想聊音乐传奇，还是你家有只叫“猫王”的喵星人呀？😺
你好！我是通义千问，英文名是Qwen。我是阿里巴巴集团旗下的通义实验室自主研发的超大规模语言模型。你可以叫我通义千问或者Qwen。有什么问题我可以帮助你吗？


测试2：

In [7]:
from langchain_core.messages import SystemMessage, HumanMessage

system_message = SystemMessage(content="我是一个人工智能助手，我的名字叫小智。")
human_message = HumanMessage(content="猫王是一只猫吗？")
human_message1 = HumanMessage(content="你叫什么名字？")

messages = [system_message, human_message, human_message1]

response = chat_model.invoke(messages)
print(response.content)

哈哈，这个问题很有趣！“猫王”通常不是指一只真正的猫，而是对美国著名摇滚歌手埃尔维斯·普雷斯利（Elvis Presley）的昵称，因为他风靡全球、魅力非凡，被粉丝们尊称为“King of Rock and Roll”（摇滚之王），中文里就亲切地叫他“猫王”。

不过，如果你家真有一只名叫“猫王”的猫咪，那它当然就是一只猫啦！🐱

至于我嘛，我的名字叫小智，很高兴认识你！有什么我可以帮你的吗？😊


测试3：

In [8]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

system_message = SystemMessage(content="我是一个人工智能助手，我的名字叫小智。")
human_message = HumanMessage(content="猫王是一只猫吗？")
system_message1 = SystemMessage(content="我可以做很多事情，有需要就找我吧。")
human_message1 = HumanMessage(content="你叫什么名字？")

messages = [system_message, human_message, system_message1, human_message1]

response = chat_model.invoke(messages)
print(response.content)

你好！我叫小智，很高兴认识你～  

至于“猫王”，其实它并不是一只猫哦！“猫王”通常是指美国传奇摇滚歌手**埃尔维斯·普雷斯利（Elvis Presley）**，因为他在舞台上的魅力和独特的表演风格被粉丝们尊称为“猫王”。虽然名字里有“猫”，但他可是人类中的超级巨星呢！🐱🎤

不过，如果你是在说某只叫“猫王”的宠物猫，那它当然就是一只可爱的猫咪啦！需要我帮你查更多关于猫王（人 or 猫？）的信息吗？或者有其他问题？随时告诉我！ 😊


测试4：

In [9]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# 第一组
system_message = SystemMessage(content="我是一个人工智能助手，我的名字叫小智。")
human_message = HumanMessage(content="猫王是一只猫吗？")

messages = [system_message, human_message]

# 第二组
system_message1 = SystemMessage(content="我可以做很多事情，有需要就找我吧。")
human_message1 = HumanMessage(content="你叫什么名字？")

messages1 = [system_message1, human_message1]

response = chat_model.invoke(messages)
print(response.content)

response1 = chat_model.invoke(messages1)
print(response1.content)

哈哈，这是个很有趣的问题！

“猫王”通常并不是指一只真正的猫，而是对美国著名摇滚歌手**埃尔维斯·普雷斯利（Elvis Presley）**的昵称。他因为独特的音乐风格、舞台魅力和标志性的发型与舞步，在20世纪50年代风靡全球，被尊称为“摇滚乐之王”（The King of Rock and Roll），中文里常亲切地叫他“猫王”。

不过，如果你是在开玩笑，或者真的在说一只名叫“猫王”的猫咪——那它当然就是一只猫啦！😺

所以，“猫王是不是一只猫”，要看语境哦～  
是人？是猫？还是你家那只自封为“王”的主子？
我叫通义千问，英文名是Qwen。我是通义实验室研发的超大规模语言模型，可以回答问题、创作文字、编程等等。有什么我可以帮你的吗？😊


# 3、关于模型调用方法的说明

invoke() / stream()

batch()：批量调用

ainvoke() / astream() / abatch()：异步调用

举例1：invoke()是阻塞式的调用

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
import os
import dotenv

# 前提，加载配置文件
dotenv.load_dotenv()

# 获取对话模型
chat_model = ChatOpenAI(
    model="qwen3-max",
    temperature=0,
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
    api_key=os.getenv("DASHSCOPE_API_KEY"),
)

messages = [HumanMessage(content="你好，请介绍一下你自己。")]

response = chat_model.invoke(messages)

print(response)

content='你好！我是通义千问（Qwen），由通义实验室研发的超大规模语言模型。我可以回答问题、创作文字，比如写故事、写公文、写邮件、写剧本、逻辑推理、编程等等，还能表达观点，玩游戏等。我支持多种语言，包括但不限于中文、英文、德语、法语、西班牙语等。\n\n如果你有任何问题或需要帮助，随时告诉我！😊' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 13, 'total_tokens': 101, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'qwen3-max', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run--1807a62c-fa6b-42be-a9da-26ecfae50ec5-0' usage_metadata={'input_tokens': 13, 'output_tokens': 88, 'total_tokens': 101, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}


举例2：流式的演示

In [16]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
import os
import dotenv

# 前提，加载配置文件
dotenv.load_dotenv()

# 获取对话模型
chat_model = ChatOpenAI(
    model="qwen3-max",
    temperature=0,
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    streaming=True,
)

messages = [HumanMessage(content="你好，请介绍一下你自己。")]

for chunk in chat_model.stream(messages):
    print(chunk.content, end="", flush=True)

你好！我是通义千问（Qwen），由阿里巴巴集团旗下的通义实验室自主研发的超大规模语言模型。我可以回答问题、创作文字，比如写故事、写公文、写邮件、写剧本、逻辑推理、编程等等，还能表达观点，玩游戏等。我支持多种语言，包括但不限于中文、英文、德语、法语、西班牙语等。

如果你有任何问题或需要帮助，随时告诉我！😊

举例三：批量调用的演示

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
import os
import dotenv

# 前提，加载配置文件
dotenv.load_dotenv()

# 获取对话模型
chat_model = ChatOpenAI(
    model="qwen3-max",
    temperature=0,
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    streaming=True,
)

messages1 = [
    SystemMessage(content="你是一个乐于助人的智能小助手。"),
    HumanMessage(content="你好，请介绍一下你自己。")
]
messages2 = [
    SystemMessage(content="你是一个乐于助人的智能小助手。"),
    HumanMessage(content="请帮我介绍一下AIGC。")
]
messages3 = [
    SystemMessage(content="你是一个乐于助人的智能小助手。"),
    HumanMessage(content="请帮我写一首关于春天的诗。")
]

batch_messages = [messages1, messages2, messages3]

responses = chat_model.batch(batch_messages)

print(responses)

[AIMessage(content='你好！我是通义千问（Qwen），阿里巴巴集团旗下的超大规模语言模型。我可以帮助你回答问题、创作文字，比如写故事、写公文、写邮件、写剧本、逻辑推理、编程等等，还能表达观点，玩游戏等。我支持多种语言，包括但不限于中文、英文、德语、法语、西班牙语等。\n\n如果你有任何问题或需要帮助，随时告诉我！😊', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'qwen3-max'}, id='run--389cb379-c1a1-4bcf-a4ad-b428bbeb364e-0'), AIMessage(content='当然可以！AIGC 是 “AI Generated Content” 的缩写，中文通常翻译为“人工智能生成内容”。它是指利用人工智能（AI）技术，特别是基于大模型（如 GPT、文心一言、通义千问、Stable Diffusion、Midjourney 等）自动生成文本、图像、音频、视频、代码等多种形式内容的技术和应用。\n\n### AIGC 的主要特点：\n\n1. **自动化生成**：无需人工逐字逐句创作，AI 可根据提示（prompt）快速生成高质量内容。\n2. **多模态支持**：不仅能生成文字，还能生成图像、音乐、语音、视频等，实现跨模态内容创作。\n3. **高效低成本**：相比传统内容创作方式，AIGC 能大幅提高效率、降低人力成本。\n4. **个性化定制**：通过调整输入提示或参数，可生成符合特定风格、语气或用途的内容。\n\n### AIGC 的典型应用场景：\n\n- **文本生成**：如撰写新闻稿、营销文案、小说、诗歌、邮件、报告、代码注释等。\n- **图像生成**：如 AI 绘画、产品设计图、广告素材、游戏角色设计等（代表工具：Midjourney、Stable Diffusion）。\n- **音视频生成**：如 AI 配音、虚拟主播、短视频自动剪辑、AI 歌曲创作等。\n- **教育与科研**：辅助教学材料编写、论文摘要生成、实验数据分析等。\n- **游戏与娱乐**：自动生成剧情、对话、地图、NPC 行为等。\n\n### AIGC 的核心技术支撑：\n\n- **大语言模型

距离四：关于同步和异步方法的调用

例子1：同步方法调用的演示

In [3]:
import time

def call_model():
    # 模拟同步API调用
    print("开始调用模型...")
    time.sleep(5)  # 模拟耗时操作
    print("模型调用完成！")

def perform_other_tasks():
    # 模拟其他任务
    for i in range(5):
        print(f"执行其他任务 {i+1}")
        time.sleep(1)

def main():
    start_time = time.time()
    call_model()
    perform_other_tasks()
    end_time = time.time()
    return f"总耗时: {end_time - start_time} 秒"

main_time = main()
print(main_time)

开始调用模型...
模型调用完成！
执行其他任务 1
执行其他任务 2
执行其他任务 3
执行其他任务 4
执行其他任务 5
总耗时: 10.060813665390015 秒


例子2：异步方法调用的演示

In [5]:
import asyncio
import time

async def async_call(llm):
    await asyncio.sleep(5)  # 模拟耗时操作
    print("模型异步调用完成！")

async def perform_other_tasks():
    await asyncio.sleep(5) # 模拟其他任务耗时操作
    print("其他任务执行完成！")

async def run_async_tasks():
    start_time = time.time()
    await asyncio.gather(
        async_call(None), # 模拟传入模型实例
        perform_other_tasks()
    )
    end_time = time.time()
    return f"总耗时: {end_time - start_time} 秒"

# 正确运行异步调用任务的方法
# if __name__ == "__main__":
#     result = asyncio.run(run_async_tasks())
#     print(result)

# 在Jupyter Notebook中运行异步调用任务的方法
result = await run_async_tasks()
print(result)

模型异步调用完成！
其他任务执行完成！
总耗时: 5.006426811218262 秒
